<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Tool Use

Tool use allows customers to **connect their large language models to external tools like search engines, APIs, functions, databases**, etc.

This allows the customer's model to unlock a richer set of behaviors by leveraging data stored in tools, taking actions through APIs, interacting with a vector database, querying a search engine, etc.

This is particularly valuable for enterprise customers, since a lot of enterprise data lives in external sources.

Tool Use consists of 4 steps:
- Step 1: the user configures the request to the model
- Step 2: the **model smartly decides which tool(s) to use and how**
- Step 3: the tool calls are executed to mock database
- Step 4: the **model generates a final answer with precise citations based on the tool results**

In [13]:
import cohere
import json

co = cohere.ClientV2("COHERE_API_KEY") # Get your free API key: https://dashboard.cohere.com/api-keys

## Step 0: Create a mock database
First, we'll define the mock data that our tools will query. This data represents sales reports and a product catalog.

In [2]:
# Mock database containing daily sales reports
sales_database = {
    '2023-09-28': {
        'total_sales_amount': 5000,
        'total_units_sold': 100,
    },
    '2023-09-29': {
        'total_sales_amount': 10000,
        'total_units_sold': 250,
    },
    '2023-09-30': {
        'total_sales_amount': 8000,
        'total_units_sold': 200,
    }
}

# Mock product catalog
product_catalog = {
    'Electronics': [
        {'product_id': 'E1001', 'name': 'Smartphone', 'price': 500, 'stock_level': 20},
        {'product_id': 'E1002', 'name': 'Laptop', 'price': 1000, 'stock_level': 15},
        {'product_id': 'E1003', 'name': 'Tablet', 'price': 300, 'stock_level': 25},
    ],
    'Clothing': [
        {'product_id': 'C1001', 'name': 'T-Shirt', 'price': 20, 'stock_level': 100},
        {'product_id': 'C1002', 'name': 'Jeans', 'price': 50, 'stock_level': 80},
        {'product_id': 'C1003', 'name': 'Jacket', 'price': 100, 'stock_level': 40},
    ]
}

Now, we'll define the tools that simulate querying this database.  
You could for example use the API of an enterprise sales platform.



In [3]:
# Function definitions
def query_daily_sales_report(day: str) -> dict:
    """
    Function to retrieve the sales report for the given day
    """
    report = sales_database.get(day, {})
    if report:
        return [{
            "date": day,
            "summary": f"Total Sales Amount: {report['total_sales_amount']}, Total Units Sold: {report['total_units_sold']}",
        }]
    else:
        return [{"date": day, "summary": "No sales data available for this day."}] 
    # You can return a list of objects e.g. [{"url": "abc.com", "text": "..."}, {"url": "xyz.com", "text": "..."}]

def query_product_catalog(category: str) -> dict:
    """
    Function to retrieve products for the given category
    """
    products = product_catalog.get(category, [])
    return [{"category": category, "products": products}]
    # You can return a list of objects e.g. [{"url": "abc.com", "text": "..."}, {"url": "xyz.com", "text": "..."}]


functions_map = {
    "query_daily_sales_report": query_daily_sales_report,
    "query_product_catalog": query_product_catalog,
}

## Step 1 - the user configures the request to the model

The developer provides a few things to the model:
- A preamble containing instructions about the task and the desired style for the output.
- The user request.
- A list of tools to the model.
- (Optionally) a chat history for the model to work with.


You can specify one or many tools to the model. Every tool needs to be described with a JSON schema, indicating the tool name, description, and parameters (code snippets below).

In our example, we provide two tools to the model: `daily_sales_report` and `product_catalog`.


In [4]:
# tool descriptions that the model has access to
# note: Cohere always adds a "directly_answer" tool under the hood, so that the model can decide to not leverage any tool, if they're not needed.
tools = [
    {
        "type": "function",
        "function": {
            "name": "query_daily_sales_report",
            "description": "Connects to a database to retrieve overall sales volumes and sales information for a given day.",
            "parameters": {
                "type": "object",
                "properties": {
                    "day": {
                        "type": "string",
                        "description": "Retrieves sales data for this day, formatted as YYYY-MM-DD."
                    }
                },
                "required": ["day"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_catalog",
            "description": "Connects to a product catalog with information about all the products being sold, including categories, prices, and stock levels.",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {
                        "type": "string",
                        "description": "Retrieves product information data for all products in this category."
                    }
                },
                "required": ["category"]
            }
        }
    }
]

Now let's define the user request.  

In our example we'll use: "Can you provide a sales summary for 29th September 2023, and also give me the details of all products in the 'Electronics' category that were sold that day, including their prices and stock levels?"

Only a langage model with Tool Use can answer this request: it requires looking up information in the right external tools (step 2), and then providing a final answer based on the tool results (step 4).

In [7]:
# System message containing instructions about the task and the desired style for the output.
system_message = """
## Task and Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

# User request
message = "Can you provide a sales summary for 29th September 2023, and also give me some details about the products in the 'Electronics' category, for example their prices and stock levels?"

## Step 2 – the model smartly decides which tool(s) to use and how
The model intelligently selects the right tool(s) to call -- and the right parameters for each tool call -- based on the content of the user message.

In [11]:
messages=[{"role": "system", "content": system_message},
          {"role": "user", "content": message}]


response = co.chat(model="command-r-plus-08-2024",
                   messages=messages,
                   tools=tools)

print("The model recommends doing the following tool calls:\n")
print("Tool plan:")
print(response.message.tool_plan, "\n")
print("Tool calls:")
for tc in response.message.tool_calls:
    print(f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}")

# append the chat history
messages.append(
    {
        "role": "assistant",
        "tool_calls": response.message.tool_calls,
        "tool_plan": response.message.tool_plan,
    }
)

The model recommends doing the following tool calls:

Tool plan:
I will first search for the sales summary for 29th September 2023. Then, I will search for the products in the 'Electronics' category. 

Tool calls:
Tool name: query_daily_sales_report | Parameters: {"day":"2023-09-29"}
Tool name: query_product_catalog | Parameters: {"category":"Electronics"}


## Step 3 – the tool calls are executed

You can then execute the appropriate calls, using the tool calls and tool parameters generated by the model.  
These tool calls return tool results that will be fed to the model in Step 4.

In [14]:
# Iterate over the tool calls generated by the model
for tc in response.message.tool_calls:
    # here is where you would call the tool recommended by the model, using the parameters recommended by the model
    tool_result = functions_map[tc.function.name](**json.loads(tc.function.arguments))
    # store the output in a list
    tool_content = []
    for data in tool_result:
        tool_content.append({"type": "document", "document": {"data": json.dumps(data)}})
        # Optional: add an "id" field in the "document" object, otherwise IDs are auto-generated
    # append the chat history
    messages.append(
        {"role": "tool", "tool_call_id": tc.id, "content": tool_content}
    )
    
    print("Tool results that will be fed back to the model in step 4:")
    for result in tool_content:
        print(result)

Tool results that will be fed back to the model in step 4:
{'type': 'document', 'document': {'data': '{"date": "2023-09-29", "summary": "Total Sales Amount: 10000, Total Units Sold: 250"}'}}
Tool results that will be fed back to the model in step 4:
{'type': 'document', 'document': {'data': '{"category": "Electronics", "products": [{"product_id": "E1001", "name": "Smartphone", "price": 500, "stock_level": 20}, {"product_id": "E1002", "name": "Laptop", "price": 1000, "stock_level": 15}, {"product_id": "E1003", "name": "Tablet", "price": 300, "stock_level": 25}]}'}}


## Step 4 - the model generates a final answer based on the tool results
Finally, the developer calls the Cohere model, providing the tools results, in order to generate the model's final answer.  

Bonus: At Cohere, all Tool Use calls come with... **precise citations**! 🎉
The model cites which tool results were used to generate the final answer.  
These citations make it easy to check where the model’s generated response claims are coming from.  
They help users gain visibility into the model reasoning, and sanity check the final model generation.  
These citations are optional — you can decide to ignore them.



In [15]:
response = co.chat(
    model="command-r-plus-08-2024",
    messages=messages,
    tools=tools
)

print("Final answer:")
print(response.message.content[0].text)

Final answer:
On 29th September 2023, the total sales amount was 10,000, and the total units sold were 250.

The following products are available in the Electronics category:
- Smartphone (E1001) - £500, 20 in stock
- Laptop (E1002) - £1,000, 15 in stock
- Tablet (E1003) - £300, 25 in stock


## Bonus: Citations come for free with Cohere! 🎉

At Cohere, model generations come with... precise citations! 🎉
The model cites which groups of words, in the tool results, were used to generate the final answer.  
These citations make it easy to check where the model’s generated response claims are coming from.  
They help users gain visibility into the model reasoning, and sanity check the final model generation.  
These citations are optional — you can decide to ignore them.

In [16]:
print("Citations that support the final answer:")
for citation in response.message.citations:
    print(f"Start: {citation.start} | End: {citation.end} | Text: '{citation.text}'")

Citations that support the final answer:
Start: 28 | End: 57 | Text: 'total sales amount was 10,000'
Start: 67 | End: 93 | Text: 'total units sold were 250.'
Start: 163 | End: 201 | Text: 'Smartphone (E1001) - £500, 20 in stock'
Start: 204 | End: 240 | Text: 'Laptop (E1002) - £1,000, 15 in stock'
Start: 243 | End: 277 | Text: 'Tablet (E1003) - £300, 25 in stock'


In [17]:
def insert_inline_citations(text, citations):
    sorted_citations = sorted(citations, key=lambda c: c.start, reverse=True)
    source_index = {}
    current_index = 1
    
    for citation in sorted_citations:
        citation_ids = []
        for source in citation.sources:
            if source.id not in source_index:
                source_index[source.id] = current_index
                current_index += 1
            citation_ids.append(str(source_index[source.id]))
        
        citation_text = f"[{','.join(citation_ids)}]"
        text = text[:citation.end] + citation_text + text[citation.end:]
    
    return text, source_index

def list_sources(citations, source_index):
    unique_sources = {}
    for citation in citations:
        for source in citation.sources:
            if hasattr(source, 'tool_output'):
                source_text = str(source.tool_output)
            else:
                source_text = str(source.document)
            unique_sources[source.id] = source_text
    
    footnotes = []
    for source_id, source_text in sorted(unique_sources.items(), key=lambda x: source_index[x[0]]):
        index = source_index[source_id]
        footnotes.append(f"[{index}]: {source_id} | {source_text}")
    
    return "Source tools:\n" + "\n".join(footnotes)

# Usage example:
cited_text, source_index = insert_inline_citations(response.message.content[0].text, response.message.citations)

print(cited_text)

print("\n" + list_sources(response.message.citations, source_index))

On 29th September 2023, the total sales amount was 10,000[2], and the total units sold were 250.[2]

The following products are available in the Electronics category:
- Smartphone (E1001) - £500, 20 in stock[1]
- Laptop (E1002) - £1,000, 15 in stock[1]
- Tablet (E1003) - £300, 25 in stock[1]

Source tools:
[1]: query_product_catalog_zeq4jk768e19:0 | {'content': '{"category": "Electronics", "products": [{"product_id": "E1001", "name": "Smartphone", "price": 500, "stock_level": 20}, {"product_id": "E1002", "name": "Laptop", "price": 1000, "stock_level": 15}, {"product_id": "E1003", "name": "Tablet", "price": 300, "stock_level": 25}]}'}
[2]: query_daily_sales_report_7me1rzpgww1x:0 | {'date': '2023-09-29', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250'}


Yiha. You've used Cohere for Tool Use.  Tool use opens up a wide range of new use cases. Here are a few examples:

- **Function calling**: It's now possible to ask the model to output a JSON object with specific function parameters.
For instance, this allows your chatbot to interact with your CRM to change the status of a deal, or to engage with a Python interpreter to conduct data science analyses.

- **Query transformation**: You can transform a user message into a search query for a vector database or any search engine.
For instance, this enables your work assistant to automatically retrieve the appropriate data from your company's documentation by creating the right query for your vector database.

- **Advanced searches**: You can transform a user message into one-or-many queries, to do multiple subtasks based on the content of the message.
For instance, this allows your chatbot to search across different databases and platforms to retrieve relevant information or to conduct comparative analysis.

